In [9]:
import pandas as pd
import numpy as np
from osgeo import gdal, osr, ogr
import sys
import math
import os
import subprocess

In [10]:
SPATIAL_IDX = 5
TEMPORAL_IDX = 9

In [11]:
'''
Open input file <filename> and return numpy array
'''

def getInputData(filename):
    # This allows gdal to throw python exceptions
    gdal.UseExceptions()
    try:
        modisRaster = gdal.Open(filename)
    except e:
        print("Unable to open tif file")
        print(e)
        print(modisRaster.GetMetadata())
    modisData = modisRaster.ReadAsArray()
    # set NaN values to 0
    modisData = np.where((modisData < 1) | (modisData > 366), 0, modisData)
    modisData = np.floor(modisData)
    return modisData,modisRaster
    

In [12]:
'''
Set the number of spatial window based on row/col and SPATIAL_IDX
'''
def setSpatialWindow(row,col,arrayDims,spatial_idx):
    top = max(0,row-spatial_idx)
    bottom = min(arrayDims[0],row+spatial_idx+1)
    left = max(0,col-spatial_idx)
    right = min(arrayDims[1],col+spatial_idx+1)
    return top,bottom,left,right
    

In [13]:
'''
Process the input array determining fire events based on SPATIAL_IDX and TEMPORAL_IDX
'''
def getEventData(inputArray,spatial_idx,temporal_idx):
    nextId = 1
    eventArray = np.zeros(inputArray.shape) #initialize event array to 0s
    #Traverse pixels in the MODIS Data Array to look for neighboring event pixels that belong to the event
    for row in range(inputArray.shape[0]):
        for col in range(inputArray.shape[1]):
            if inputArray[row,col] : # burn detected
                #print("{},{} burn detected".format(row,col))
                burnDay = inputArray[row,col]
                #print("Next ID={}, burnDay={}:".format(nextId,burnDay))
                # set spatial input and temporal bounds and create spatial windows
                top,bottom,left,right = setSpatialWindow(row,col,inputArray.shape,spatial_idx)
                inputWindow = inputArray[top:bottom,left:right]
                temporalMask = np.logical_and(inputWindow>0,abs(inputWindow-burnDay)<=temporal_idx)
                eventWindow = eventArray[top:bottom,left:right]
                
                currId = eventArray[row,col] # Event ID for current pixel
                overlapMask = np.logical_and(np.logical_and(eventWindow>0,temporalMask),eventWindow!=currId)
                if overlapMask.any():
                    #true overlap
                    overlapId = np.amin(eventWindow[overlapMask],axis=None)
                    eventWindow[temporalMask] = overlapId
                    if currId != 0:
                        #print("Event merge: setting {} to {}".format(currId,overlapId))
                        eventArray[eventArray==currId] = overlapId
                elif currId == 0: # no overlap and not set so get next ID
                    eventWindow[temporalMask] = nextId
                    nextId += 1
                else: 
                    eventWindow[temporalMask] = currId # propagate current id
    return eventArray

In [14]:
'''
Save the event data to GTiff file
'''
def saveEventData(eventData,input_raster,outfile):
    print("Saving to {}".format(outfile))
    rows = input_raster.RasterYSize
    cols = input_raster.RasterXSize
    output_raster = gdal.GetDriverByName('GTiff').Create(outfile, cols, rows, 1 ,gdal.GDT_Float32)
    output_raster.SetGeoTransform(input_raster.GetGeoTransform())
    output_raster.SetProjection(input_raster.GetProjection())
    output_raster.GetRasterBand(1).WriteArray(eventData) 

In [15]:
'''
Calculate event statistics for each fire event
'''
def getEventStats(eventData):
    eventid, pixels = np.unique(eventData, return_counts=True)
    df=pd.DataFrame({'eventID':eventid, 'pixels':pixels})
    df = df[df.eventID != 0] # drop '0
    df['km2'] = (df.pixels * 463*463)/1000000
    df['firstbd'] = df.eventID.apply(lambda x: modisData[eventData==x].min())
    df['lastbd'] = df.eventID.apply(lambda x: modisData[eventData==x].max())
    df['duration'] = df.lastbd - df.firstbd + 1
    df['fsr'] = df.pixels/df.duration
    df['maxbd'] = df.eventID.apply(lambda x: np.bincount(modisData[eventData==x].astype(int)).argmax())
    df_e_all = []
    events = eventid[eventid>0] #exclude pixels set to 0 (no event)
    for e in events:
        bd,bd_pixels = np.unique(modisData[eventData==e],return_counts=True)
        df_e = pd.DataFrame({'eventID':e,'bd':bd,'pixels':bd_pixels})
        df_e['areakm2'] = (df_e.pixels * 463*463)/1000000
        df_e_all.append(df_e)
    df_detail = pd.concat(df_e_all)
    return df,df_detail


In [16]:
def processFireEvent(infile,output_base,s,t):
    inputData,inputRaster = getInputData(infile)
    eventData = getEventData(inputData,s,t)
    saveEventData(eventData,inputRaster,(output_base + '.tif'))

In [17]:
def arraytoCSV(a,filename):
    df = pd.DataFrame(a)
    df.to_csv(filename)

In [ ]:
input_path = '../data/MCD64A1/C6/yearly_composites/'
output_path = '../data/MCD64A1/C6/yearly_composites_15x15/'

for s in range(1,16): # 1 to 15
    for t in range(1,16): # 1 to 15
        for year in range(2001,2018):
            if not os.path.exists(output_path + 'USA_BurnDate_' + str(year) + 's' + str(s) + 't' + str(t) + '.tif'):
                print("s={} t={} year={}".format(s,t,year))
                infile = input_path + 'USA_BurnDate_' + str(year) + '_ms.tif'
                output_base =  output_path + 'USA_BurnDate_' + str(year) + 's' + str(s) + 't' + str(t) 
                processFireEvent(infile,output_base,s,t)
                subprocess.call(['aws', 's3', 'sync', output_path, 
                                 's3://earthlab-natem/modis-burned-area/MCD64A1/C6/yearly_composites_15x15'])
                os.remove(output_path + 'USA_BurnDate_' + str(year) + 's' + str(s) + 't' + str(t) + '.tif')
            else: 
                print("deleting... s={} t={} year={}".format(s,t,year))
                os.remove(output_path + 'USA_BurnDate_' + str(year) + 's' + str(s) + 't' + str(t) + '.tif')